In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from geopy import Nominatim


import os
import urllib.request
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import spacy

from sklearn.cluster import KMeans


In [2]:
ratings = pd.read_csv('../data/chocolate_ratings.csv')
ratings.shape, ratings.columns

((2530, 10),
 Index(['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date',
        'Country of Bean Origin', 'Specific Bean Origin or Bar Name',
        'Cocoa Percent', 'Ingredients', 'Most Memorable Characteristics',
        'Rating'],
       dtype='object'))

In [3]:
ratings['Cocoa Percent Int'] = ratings['Cocoa Percent'].str[:2].astype(int)

abbrev2ingredient = {'B':'Beans', 'S':'Sugar', 'S*': 'Sweetener other than white cane or beet sugar', 'C':'Cocoa Butter', 
                    'V': 'Vanilla', 'L': 'Lecithin', 'Sa': 'Salt', '':''}

ratings['Ingredients List'] = [[abbrev2ingredient[x] for x in i[1].strip().split(',')] for i in ratings['Ingredients'].fillna('0 - ').str.split('-')]

ratings['Most Memorable Characteristics List'] = ratings['Most Memorable Characteristics'].fillna('none,').str.split(',')

In [4]:
ratings.head()

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Cocoa Percent Int,Ingredients List,Most Memorable Characteristics List
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25,76,"[Beans, Sugar, Cocoa Butter]","[rich cocoa, fatty, bready]"
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50,76,"[Beans, Sugar, Cocoa Butter]","[cocoa, vegetal, savory]"
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75,76,"[Beans, Sugar, Cocoa Butter]","[cocoa, blackberry, full body]"
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68%,"3- B,S,C","chewy, off, rubbery",3.00,68,"[Beans, Sugar, Cocoa Butter]","[chewy, off, rubbery]"
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72%,"3- B,S,C","fatty, earthy, moss, nutty,chalky",3.00,72,"[Beans, Sugar, Cocoa Butter]","[fatty, earthy, moss, nutty, chalky]"


In [5]:
ratings.explode('Ingredients List')['Ingredients List'].unique()

array(['Beans', 'Sugar', 'Cocoa Butter', 'Lecithin', 'Vanilla', 'Salt',
       '', 'Sweetener other than white cane or beet sugar'], dtype=object)

In [6]:
flavours = list(ratings.explode('Most Memorable Characteristics List')['Most Memorable Characteristics List'].unique())
len(flavours)

1232

In [7]:
import spacy
nlp = spacy.load('en_core_web_md')

flavour_embds = [np.mean([token.vector for i,token in enumerate(nlp(doc)) if (token.pos_ in ["ADJ", "NOUN"]) or i == len(nlp(doc))-1], axis=0) for doc in tqdm(flavours)]

flavour_embds[0]

 51%|█████     | 630/1232 [00:13<00:13, 45.28it/s]C:\Users\ludov\anaconda3\envs\ada\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ludov\anaconda3\envs\ada\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1232/1232 [00:27<00:00, 45.55it/s]


array([-1.54005006e-01, -1.20966502e-01,  1.83849990e-01, -1.02813005e-01,
       -4.07429993e-01,  3.45719993e-01,  1.65680006e-01,  1.55233502e-01,
       -1.33925512e-01,  1.16965997e+00, -5.84129989e-01, -8.59294981e-02,
       -2.24844992e-01,  1.75199956e-02,  1.12769991e-01, -3.26214999e-01,
       -7.49095008e-02,  1.09688497e+00,  2.94016480e-01, -4.75584984e-01,
       -2.67454982e-01, -1.00549981e-02,  8.45865011e-02,  3.93880010e-01,
       -5.78589998e-02, -4.90425020e-01,  2.91509986e-01,  4.49180007e-01,
        8.52850080e-02, -6.94689989e-01,  4.99188900e-06,  1.23570003e-01,
        1.85879007e-01, -4.71915007e-01,  1.16549999e-01, -2.26609007e-01,
       -3.55574995e-01, -2.99239993e-01, -1.54258996e-01, -1.09274998e-01,
        2.26435006e-01, -4.00165021e-01,  9.43975002e-02, -2.96508998e-01,
        7.54429996e-02,  4.18060005e-01,  1.15400031e-02, -8.18825066e-02,
        2.17345998e-01, -2.83675015e-01,  8.68194997e-02,  2.12384984e-01,
       -1.85939997e-01, -

In [8]:
flavour_embds = [f_emb if type(f_emb) == np.ndarray else nlp('none').vector for f_emb in flavour_embds]

In [9]:
kmeans = KMeans(n_clusters=200, random_state=0).fit(np.array(flavour_embds))

In [10]:
flavour_labels = kmeans.labels_

In [11]:
from sklearn.metrics import pairwise_distances_argmin_min
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, np.array(flavour_embds))
closest

array([  41,  477,  565,  414,  974,  101,   52,  525,  823,  199,    8,
          2,   43,  640,   21,   12,  572,  170, 1006,   76,  128,   42,
         19,  112,   25,   88,  227,  390,    3,   45,   57,   93,    4,
         31,   58,  242,   16,  435,  862,  237,  786,   15,    5,   39,
        447,  210,  436, 1228,  693,    6,   40,   32,  822,   49,   18,
         44,  251, 1034,  120,  694,   70,   59,  835,  276,   77,  129,
         84,  941,  938,    0,  158,  438,  295,  167,   83,  726,  195,
        100,   87,  658,  532,   27,  384, 1087,   69,   48,  252,   98,
         61,  774,  720,  102,   46,   85,  200,  118,  567,   89,  256,
         99,   10,  185,  229,  459,  161,  825,   68,  587,  193,  141,
        377,  179,  153,  172,  391,  336,  428,  325,   94,  513, 1050,
        107,  353,  207, 1195, 1008,  764,   74,  443,  563,  622,  512,
        783,  351,  105,  644,   86,  835,  280,  440,  181,  932,   55,
        602,  216,  612,  253,  713,  509,  762,   

In [12]:
centers = np.array(flavours)[closest]

centers

array(['oily', 'full flavor w/ dominant roast', 'sweet spice',
       ' hint of fruit', 'sour banana', ' ashey', ' mild choco',
       ' basic cocoa base', 'mild bitter and chemical off', 'dark berry',
       'chewy', ' bready', ' caramel', ' metal', 'fruity', ' earthy',
       ' odd rubber notes', ' burnt', ' slight woody', 'herbal',
       ' cinamon', ' nut', ' macadamia', 'grit', 'burnt rubber',
       ' metallic', ' black licorice', ' soil', 'cocoa', 'sweet',
       ' brownie', ' coffee', ' vegetal', ' grass', 'burnt wood', 'honey',
       'mildly bitter', ' muted', ' dirty/burnt edge', 'strawberries',
       ' cheesey', 'chalky', ' savory', 'sandy', ' slight cherry',
       'smooth', ' grain', ' intense blueberry', 'slow develop',
       ' blackberry', ' fig', 'mild tobacco', 'cooling sensation',
       'sliglty dry', 'milk brownie', ' raspberry', ' ham',
       'bright red fruit', ' mint', ' mild fatty', 'light color',
       ' choco', 'mild vanilla and mild fruit', 'pastey', ' m

In [13]:
data = pd.DataFrame({'flavour':flavours, 'embedding':flavour_embds, 'group':flavour_labels, 'group_name':[centers[f_label] for f_label in flavour_labels]})

data.sample(10)

,flavour,embedding,group,group_name
776,harsh flavor,"[-0.417225, 0.37487, 0.024389997, -0.130381, -...",119,harsh aroma
784,slight musty,"[-0.1492, -0.0749, -0.403885, 0.01602, 0.01579...",150,musty
127,red berry,"[-0.430745, 0.16678998, 0.1396825, -0.272735, ...",9,dark berry
1143,sour malt,"[-0.39265, 0.27447, 0.014615003, -0.0099559985...",107,very roasty
850,mild honey,"[-0.315835, 0.299995, -0.168485, 0.096395, -0....",62,mild vanilla and mild fruit
601,unpleasant,"[-0.22948, 0.085832, -0.29603, 0.13993, -0.178...",8,mild bitter and chemical off
715,bold olive,"[0.16063699, -0.0618145, 0.003969997, -0.01708...",133,bold
30,smokey,"[0.20154, 0.6276, -0.30037, 0.07283, 0.21922, ...",15,earthy
780,high acid,"[-0.649795, 0.688155, -0.2452155, 0.0024449974...",154,high astringnet
686,mild smoke and fruit,"[-0.051093336, 0.17311001, -0.027131, -0.24148...",62,mild vanilla and mild fruit


In [14]:
centers = []
for group, flavour in data.groupby('group')['flavour']:
    centers.append(list(flavour)[np.argmin(np.array(flavour.str.len().values))].strip())

centers

['oily',
 'classic',
 'spice',
 'ripe',
 'banana',
 'wtf',
 'mild tart',
 'deep cocoa',
 'mild off',
 'red berry',
 'chewy',
 'malt',
 'tart',
 'metal',
 'spicy',
 'smokey',
 'odd',
 'burnt',
 'dark woody',
 'herbs',
 'anise',
 'nut',
 'almond',
 'grit',
 'rubber',
 'metallic',
 'black tea',
 'soil',
 'cocoa',
 'sweet',
 'fudgey',
 'coffee',
 'woody',
 'grass',
 'wood',
 'honey',
 'bitter',
 'muted',
 'deep',
 'cherries',
 'cheesy',
 'chalky',
 'meaty',
 'sandy',
 'deep cherry',
 'silky',
 'grain',
 'intense tart',
 'slow',
 'apple',
 'fig',
 'cigar',
 'cooling sensation',
 'dry',
 'pudding',
 'licorice',
 'ham',
 'green',
 'mint',
 'fat',
 'bright',
 'choco',
 'mild ham',
 'pastey',
 'molasses',
 'strong',
 'floral',
 'smoke',
 'thick',
 'rich cocoa',
 'nuts',
 'sharp',
 'rustic',
 'grainy',
 'orange',
 'accesible',
 'subtle',
 'dirt',
 'oats',
 'plums',
 'salt',
 'sticky',
 'fuel',
 'tart fruit',
 'dairy',
 'leather',
 'dull',
 'hay',
 'acidic',
 'palm',
 'undefined',
 'burlap',
 'pl

In [15]:
data['group_name'] = data['group'].apply(lambda x: centers[x])

In [16]:
for g, d in data.groupby('group_name'):
    print(g, '\n', d['flavour'].sample(3, replace=True))

 
 1060           refined
7            full body
633     low refinement
Name: flavour, dtype: object
accesible 
 726    accessible
726    accessible
920     accesible
Name: flavour, dtype: object
acidic 
 328     acidic
61      acidic
328     acidic
Name: flavour, dtype: object
almond 
 1026     pistachio
56        hazelnut
1171       walnuts
Name: flavour, dtype: object
alocohol 
 356     alocohol
356     alocohol
356     alocohol
Name: flavour, dtype: object
ambiguous 
 727     ambiguous
727     ambiguous
727     ambiguous
Name: flavour, dtype: object
anise 
 358                    raisins
624    blackberry and cinnamon
788                   cardamon
Name: flavour, dtype: object
apple 
 926      apple
1113     apple
1113     apple
Name: flavour, dtype: object
baked 
 336     bread
336     bread
336     bread
Name: flavour, dtype: object
balanced 
 201           balanced
851      well balanced
1149     well balanced
Name: flavour, dtype: object
banana 
 535        sour cherry
814     

In [17]:
new_names = ['nuts', 'other', 'burnt', 'caramel, marshmellow, nutella', 'nuts', 'cardboard', 'roasty, bitter', 'tobacco', 'berries', 'tea', 'fruit', 'cocoa', 'spicy, nutty', 'roast', 'herbal', 'mild', 'gummy', 'berries', 'sweet']


In [18]:
flavour2group = {}

for row in data.iterrows():
    flavour2group[row[1]['flavour']] = row[1]['group_name']


In [19]:
group2emb = {}

for cent in centers:
    group2emb[cent] = nlp(cent).vector

In [20]:
ratings['New Flavours'] = [[flavour2group[flav] for flav in flavors] for flavors in ratings['Most Memorable Characteristics List']]

In [21]:
ratings['Flavours Vectors'] = [np.mean(np.array([group2emb[flav.strip()] for flav in flavors]), axis=0) for flavors in ratings['New Flavours']]

In [22]:
ratings

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Cocoa Percent Int,Ingredients List,Most Memorable Characteristics List,New Flavours,Flavours Vectors
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25,76,"[Beans, Sugar, Cocoa Butter]","[rich cocoa, fatty, bready]","[rich cocoa, fat, malt]","[-0.435315, 0.15850784, -0.04282667, 0.1234329..."
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50,76,"[Beans, Sugar, Cocoa Butter]","[cocoa, vegetal, savory]","[cocoa, woody, meaty]","[-0.22953333, 0.136224, -0.11319, -0.09985, -0..."
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75,76,"[Beans, Sugar, Cocoa Butter]","[cocoa, blackberry, full body]","[cocoa, apple, ]","[-0.068569995, 0.12159232, -0.01899, -0.137396..."
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68%,"3- B,S,C","chewy, off, rubbery",3.00,68,"[Beans, Sugar, Cocoa Butter]","[chewy, off, rubbery]","[chewy, off, rubbery]","[0.28423834, -0.035696674, -0.14546233, 0.2694..."
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72%,"3- B,S,C","fatty, earthy, moss, nutty,chalky",3.00,72,"[Beans, Sugar, Cocoa Butter]","[fatty, earthy, moss, nutty, chalky]","[fat, smokey, moss, chewy, chalky]","[0.084489994, 0.30870998, -0.25795, 0.19503121..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525,1205,Zotter,Austria,2014,Blend,Raw,80%,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75,80,"[Beans, Sweetener other than white cane or bee...","[waxy, cloying, vegetal]","[oily, sugary, woody]","[-0.08211667, 0.013276677, -0.28126234, 0.0032..."
2526,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75%,"3- B,S,C","strong nutty, marshmallow",3.75,75,"[Beans, Sugar, Cocoa Butter]","[strong nutty, marshmallow]","[strong, choco]","[-0.100362, 0.275335, 0.013254993, 0.009725004..."
2527,2036,Zotter,Austria,2018,Blend,"Dry Aged, 30 yr Anniversary bar",75%,"3- B,S,C","fatty, earthy, cocoa",3.00,75,"[Beans, Sugar, Cocoa Butter]","[fatty, earthy, cocoa]","[fat, smokey, cocoa]","[-0.19786, 0.27682897, -0.11371335, 0.14996, -..."
2528,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70%,"3- B,S,C","fatty, mild nuts, mild fruit",3.25,70,"[Beans, Sugar, Cocoa Butter]","[fatty, mild nuts, mild fruit]","[fat, mild ham, mild ham]","[-0.62492675, 0.14060333, 0.06380334, 0.562856..."


In [23]:
ratings.to_json('postprocessing_form.json')

In [24]:
ratings.T.to_json('postprocessing_form_t.json')

In [27]:
pd.DataFrame({'flavour':group2emb.keys(), 'vector':group2emb.values()}).set_index('flavour').T.to_json('group2emb.json')